# Node
Node is the basic building block of the WorkGraph. A node has inputs, outputs, and the executor. A node executor can be a `calcfunction`, `workfunction`, `calcjob`, `Workchain` or any other Python function. A node can be created in three ways.

## Decorator

Decorate any Python function using the `node` decorator. To use the power of AiiDA (e.g. save the results to a database, keep provenance), one can use the `node.calcfunction` decorator.


In [1]:
from aiida_workgraph import node
from aiida import orm

# define add node
@node()
def add(x, y):
   return x + y

# define multiply calcfunction node
@node.calcfunction()
def multiply(x, y):
   return orm.Float(x + y)

The input sockets are generated automatically based on the function arguments. The default name of the output socket is `result`. There are also some built-in sockets, like `_wait` and `_outputs`.  One can create a node instance and inpsect its inputs and outputs:

In [2]:
add1 = add.node()
print("Inputs:", add1.inputs.keys())
print("Outputs:", add1.outputs.keys())

Inputs: ['x', 'y', '_wait']
Outputs: ['result', '_wait', '_outputs']


If you want to change the name of the output sockets, or if there are more than one output. You can define the outputs explicitly.
For example: `["General", "sum"]`, where the first value `General` indicates the data type, and the second value is the name of the socket. The data type tell the code how to display the socket in the GUI and to serialize data into database. We use `General` for any data type.

In [3]:
# define add calcfunction node
@node(outputs=[["General", "sum"],
               ["General", "difference"]])
def add_minus(x, y):
   return {"sum": x + y, "difference": x - y}

print("Inputs:", add_minus.node().inputs.keys())
print("Outputs:", add_minus.node().outputs.keys())

Inputs: ['x', 'y', '_wait']
Outputs: ['sum', 'difference', '_wait', '_outputs']


Then, one can use the node inside the WorkGraph:

In [4]:
from aiida_workgraph import WorkGraph
wg = WorkGraph()
add_minus1 = wg.nodes.new(add_minus, name="add_minus1")
multiply1 = wg.nodes.new(multiply, name="multiply1")
wg.links.new(add_minus1.outputs["sum"], multiply1.inputs["x"])

NodeLink(from="add_minus1.sum", to="multiply1.x")

## Build from Callable

One can build a node from an already existing Python function.

In [5]:
from aiida_workgraph import WorkGraph, build_node

from scipy.linalg import norm
NormNode = build_node(norm)

wg = WorkGraph()
wg.nodes.new(NormNode, name="norm1")


NodeGraphWidget(settings={'minmap': False}, style={'width': '40%', 'height': '600px'}, value={'nodes': {'norm1…

The inputs and outputs of the node are automatically generated based on the AiiDA process.

In [7]:
node = NormNode()
print("Inputs:")
for input in node.inputs:
   if "." not in input.name:
      print(f"  - {input.name}")
print("Outputs:")
for output in node.outputs:
   if "." not in output.name:
      print(f"  - {output.name}")

Inputs:
  - a
  - ord
  - axis
  - keepdims
  - check_finite
  - _wait
Outputs:
  - result
  - _wait
  - _outputs


One can use these AiiDA component direclty in the WorkGraph. The inputs and outputs of the node is automatically generated based on the input and output port of the AiiDA component.

In [ ]:
from aiida.calculations.arithmetic.add import ArithmeticAddCalculation

wg = WorkGraph()
add1 = wg.nodes.new(ArithmeticAddCalculation, name="add1")

## Define a Node
Create a node class by inheriting from `Node` base class.


In [ ]:
from aiida_workgraph.node import Node

class MyAdd(Node):

    identifier: str = "MyAdd"
    name = "MyAdd"
    node_type = "calcfunction"
    catalog = "Test"
    kwargs = ["x", "y"]

    def create_sockets(self):
        self.inputs.clear()
        self.outputs.clear()
        inp = self.inputs.new("General", "x")
        inp.add_property("General", "x", default=0.0)
        inp = self.inputs.new("General", "y")
        inp.add_property("General", "y", default=0.0)
        self.outputs.new("General", "sum")

    def get_executor(self):
        return {
            "path": "aiida_workgraph.test",
            "name": "add",
        }

Then, one can use the node by using its identifier.

In [ ]:
from aiida_workgraph import WorkGraph
wg = WorkGraph()
wg.nodes.new(MyAdd, name="add1")


One can also register the node in node pool, and then use its `identifer` directly.
```python
wg.nodes.new("MyAdd", name="add1")
```